In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

Chrome_options = Options()
Chrome_options.add_argument("--verbose")
Chrome_options.add_argument('--no-sandbox')
Chrome_options.add_argument("--headless=new")
Chrome_options.add_argument("--disable-gpu")
Chrome_options.add_argument("--windows-size=1920,1200")
Chrome_options.add_argument("--disable-dev-shm-usage")
Chrome_options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=Chrome_options)

In [24]:
def safe_find_element(driver, by, value):
    try:
        return driver.find_element(by, value)
    except NoSuchElementException:
        return None

def news_scraping(news_url, driver):
    # 언론사
    press_element = safe_find_element(driver, By.XPATH, '//*[@id="ct"]/div[1]/div[1]/a/img[2]')
    press = press_element.get_attribute('title') if press_element else ""

    # 기사 제목
    title_element = safe_find_element(driver, By.ID, 'title_area')
    title = title_element.text if title_element else ""

    # 발행일자
    date_time_element = safe_find_element(driver, By.XPATH, '//*[@id="ct"]/div[1]/div[3]/div[1]/div[2]/span')
    date_time = date_time_element.text if date_time_element else ""

    # 기자
    repoter_element = safe_find_element(driver, By.XPATH, '//*[@id="JOURNALIST_CARD_LIST"]/div[1]/div/div[1]/div/div/div[1]/a[2]/span/span/em')
    repoter = repoter_element.text if repoter_element else ""

    # 기사 본문
    article_element = safe_find_element(driver, By.ID, 'dic_area')
    article = article_element.text.replace("\n", "").replace("\t", "") if article_element else ""

    # 기사 반응: 쏠쏠정보
    useful_element = safe_find_element(driver, By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[1]/a/span[2]')
    useful = useful_element.text if useful_element else ""

    # 기사 반응: 흥미진진
    wow_element = safe_find_element(driver, By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[2]/a/span[2]')
    wow = wow_element.text if wow_element else ""

    # 기사 반응: 공감백배
    touched_element = safe_find_element(driver, By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[3]/a/span[2]')
    touched = touched_element.text if touched_element else ""

    # 기사 반응: 분석탁월
    analytical_element = safe_find_element(driver, By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[4]/a/span[2]')
    analytical = analytical_element.text if analytical_element else ""

    # 기사 반응: 후속강추
    recommend_element = safe_find_element(driver, By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[5]/a/span[2]')
    recommend = recommend_element.text if recommend_element else ""

    print("뉴스:", [title, press, date_time, repoter, article, useful, wow, touched, analytical, recommend, news_url])

    return [title, press, date_time, repoter, article, useful, wow, touched, analytical, recommend, news_url]

def scraping(list_url):
    driver.implicitly_wait(3)

    news_idx = 1
    news_df = pd.DataFrame(columns = ("Title", "Press", "DateTime", "Repoter", "Article", "Useful", "Wow", "Touched", "Analytical", "Recommend", "URL"))

    for url in list_url:
        driver.get(url)
        news_df.loc[news_idx] = news_scraping(url, driver)
        news_idx += 1

    driver.close()

    return news_df

In [25]:
def make_pg_num(num):
    """Calculate the page number in the format required by the website."""
    return num if num == 1 else num+9*(num-1)

def create_url(search, page_num):
    """Create a URL with the search term and page number."""
    return f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search}&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=17&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start={page_num}"

def make_urls(search, start_pg, end_pg):
    """Generate the URLs for the range of pages."""
    return [create_url(search, make_pg_num(i)) for i in range(start_pg, end_pg+1)]

def input_with_validation(prompt):
    """Ask for input with the given prompt, repeating until a valid integer is provided."""
    while True:
        try:
            return int(input(prompt))
        except ValueError:
            print("Invalid input, please enter an integer.")

def main():
    search = input("검색 키워드를 입력해주세요: ")

    start_pg = input_with_validation("\n크롤링 시작 페이지를 입력해주세요. ex)1(숫자만 입력): ")
    print(f"\n크롤링 시작 페이지: {start_pg}페이지")

    end_pg = input_with_validation("\n크롤링 종료 페이지를 입력해주세요. ex)1(숫자만 입력): ")
    print(f"\n크롤링 종료 페이지: {end_pg}페이지")

    return make_urls(search, start_pg, end_pg)

if __name__ == "__main__":
    search_urls = main()
    print("생성된 URL: ", search_urls)


크롤링 시작 페이지: 1페이지

크롤링 종료 페이지: 2페이지
생성된 URL:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=제주도&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=17&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=제주도&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=17&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start=11']


In [26]:
# Initialize the list to store the links
list_url = []

# Iterate over the URLs
for url in search_urls:
    # Send GET request to the web page
    response = requests.get(url)

    # If the request is successful, extract the HTML content and create a BeautifulSoup object
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        links = soup.select("a.info")

        # Filter and save the links with "naver.com" in their address
        for link in links:
            href = link.get("href")
            if "naver.com" in href:
                list_url.append(href)
    else:
        print("The request failed.")

    # Sleep for 1 second
    time.sleep(2)

In [27]:
scraping(list_url)

뉴스: ["제주도 '버스·택시 무료 혜택' 읍·면 65세 이상으로 확대", '연합뉴스', '2023.06.22. 오전 10:18', '고성식 기자', '동 지역은 70세 이상…1만3천여명 추가 혜택(제주=연합뉴스) 고성식 기자 = 제주에서 무료로 버스와 택시를 탈 수 있는 대상이 읍·면 지역의 경우 65세 이상으로 확대된다.동 지역 거주자는 현재처럼 70세 이상이 무료로 이용할 수 있다.어르신 행복택시 (CG)[연합뉴스TV 제공]제주도는 다음 달 1일부터 대중교통 무료 이용이 가능한 제주교통복지카드와 어르신 행복택시 지원 대상을 기존 70세 이상에서 읍·면 지역에 한해 65세 이상으로 확대한다고 22일 밝혔다.이에 따라 도내 읍·면 지역에 주소를 둔 65세 이상 어르신이면 제주교통복지카드 신규 발급해 도내 버스를 무료로 이용할 수 있게 된다.\'어르신 행복택시\'는 읍·면 지역 65세 이상 어르신이면 1일 2회, 1일 최대 1만5천원, 연 16만8천원 범위에서 무료로 이용할 수 있다.이에 따라 대중교통 지원 대상은 기존 도내 전 지역 거주 70세 이상 어르신 8만2천467명에 읍·면 거주 65세 이상 어르신 1만3천478명이 추가됐다.제주교통복지카드는 27일부터 도내 모든 농협 영업점을 방문해 신청할 수 있다.신분증, 증명사진, 주민등록등본 및 대상자 확인 증명서(장애인복지카드, 국가유공자증 등)를 구비하고 본인이 직접 방문 신청해야 한다.이상헌 제주도 교통항공국장은 "교통복지카드 및 어르신 행복택시 확대 운영으로 어르신들의 교통편의와 접근성 향상에 크게 기여할 것으로 기대한다"고 말했다.koss@yna.co.kr', '0', '0', '0', '0', '0', 'https://n.news.naver.com/mnews/article/001/0014018279?sid=102']
뉴스: ['“제주도에서 공직생활 어때요?” 6개 기관 채용 나선다', '매일경제', '2023.06.21. 오후 2:23', '송은범 기자', '제주개발공사·에너지공사 등 6곳45명 모집

,Title,Press,DateTime,Repoter,Article,Useful,Wow,Touched,Analytical,Recommend,URL
1,제주도 '버스·택시 무료 혜택' 읍·면 65세 이상으로 확대,연합뉴스,2023.06.22. 오전 10:18,고성식 기자,동 지역은 70세 이상…1만3천여명 추가 혜택(제주=연합뉴스) 고성식 기자 = 제주...,0,0,0,0,0,https://n.news.naver.com/mnews/article/001/001...
2,“제주도에서 공직생활 어때요?” 6개 기관 채용 나선다,매일경제,2023.06.21. 오후 2:23,송은범 기자,제주개발공사·에너지공사 등 6곳45명 모집… 내달 12일까지 접수제주도청 전경.[자...,2,0,0,0,0,https://n.news.naver.com/mnews/article/009/000...
3,"제주도 수학여행 다녀온 85명, 코로나 집단감염",세계일보,,김현주 기자,"학교 측, 확진자들에게 5일간 집에 머물게 하면서 원격수업 받게 해기사 특정내용과 ...",5,0,1,0,2,https://n.news.naver.com/mnews/article/022/000...
4,"제주도·JDC, KAIST 글로벌캠퍼스 유치 추진",뉴시스,,이정민 기자,20일 ‘정례협의회’서 공공 노력 합의3개 기관 간 실무협의 통해 세부사항 논의[제...,0,0,0,0,0,https://n.news.naver.com/mnews/article/003/001...
5,"제주도, 내년도 주민참여예산 사업 선정…현장 주민투표 진행",KBS,2023.06.22. 오전 8:35,나종훈 기자,"제주도, 내년도 주민참여예산 사업 선정…현장 주민투표 진행KBS뉴스|재생 수9200...",0,0,0,0,0,https://n.news.naver.com/mnews/article/056/001...
6,,,,,,,,,,,https://n.news.naver.com/mnews/article/408/000...
7,제주도 바가지 없어졌나? 놀라운 최근 제주 렌터카 비용 상황,매일경제,,,"렌터카 플랫폼 카모아, 제주·부산·여수 順 발표제주, 중형 차량·내륙, 준중형 차량...",51,3,10,0,9,https://n.news.naver.com/mnews/article/009/000...
8,"제주도·국가인권위, 인권증진·인권문화 확산 협력 협약 체결",뉴스1,2023.06.21. 오후 3:01,강승남 기자,오영훈 제주지사와 송두환 국가인권위원장이 21일 도청에서 '제주도민 인권 증진을 위...,0,0,0,0,0,https://n.news.naver.com/mnews/article/421/000...
9,"""사회협약위 제2공항 검증 요구에도 제주도는 벙어리""",JIBS,2023.06.21. 오후 1:28,정용기 기자,제주제2공항강행저지비상도민회의 오늘(21일) 기자회견제2공항 반대 단체가 오늘(21...,3,2,0,0,0,https://n.news.naver.com/mnews/article/661/000...
10,"제주도, 농민수당 지급 범위 확대…대상자 추가 접수",KBS,2023.06.22. 오전 10:44,나종훈 기자,"제주도, 농민수당 지급 범위 확대…대상자 추가 접수KBS뉴스|재생 수9600:00 ...",0,0,0,0,0,https://n.news.naver.com/mnews/article/056/001...
